# Exemplo 9: Energia - Smart Grid Anomaly Detection (Spark)

Este notebook demonstra a detecção de anomalias em uma rede elétrica inteligente (Smart Grid) em tempo real.

**Cenário**: Monitorar a voltagem em transformadores. Picos acima de 240V ou quedas abaixo de 110V são anomalias.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic energy-grid --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Simulador de Sensores de Energia

In [ ]:
import json
import time
import random
from kafka import KafkaProducer
import threading

def generate_energy_data():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    transformers = [f'TR-{i}' for i in range(1, 11)]
    
    try:
        for _ in range(200):
            for tr in transformers:
                # 5% de chance de anomalia
                if random.random() < 0.05:
                    voltage = random.choice([random.randint(90, 109), random.randint(241, 260)])
                else:
                    voltage = random.randint(110, 240)
                    
                data = {'transformer_id': tr, 'voltage': voltage, 'timestamp': time.time()}
                producer.send('energy-grid', value=data)
            time.sleep(0.5)
    finally:
        producer.close()

thread = threading.Thread(target=generate_energy_data)
thread.start()

## 4. Detecção de Anomalias

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("SmartGrid").getOrCreate()

schema = StructType([
    StructField("transformer_id", StringType()),
    StructField("voltage", IntegerType()),
    StructField("timestamp", FloatType())
])

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "energy-grid") \
    .load()

grid_data = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Filtrar anomalias
anomalies = grid_data.filter((col("voltage") > 240) | (col("voltage") < 110))

query = anomalies.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 kafka_consumer.py